# **IMPORT LIBRERIE**

In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np

from enum import Enum

# **CONNESSIONE A CARLA**

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# **VARIABILI GLOBALI**

In [3]:
# Instanza mondo
world = client.get_world()

In [4]:
# Enum state del sistema adas
class Fcw_state(Enum):
    IDLE = 4
    WARNING = 3
    ACTION = 2
    ESCAPE = 1

In [5]:
# Variabili di configurazione
min_fcw_state = Fcw_state.IDLE
vehicle_vertical_dimension = 0.75
vehicle_lateral_dimension = 0.9
min_ttc = 0.5
average_reaction_time = 2.5
throttle = 0.7
velocity_threshold = 2.77
asphalt_friction_coefficient = 0.8
asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient

# **METODI PER LA CREAZIONE DI ATTORI**

In [6]:
def spawn_vehicle(vehicle_index=10, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [7]:
def spawn_radar(vehicle):
    rad_bp = world.get_blueprint_library().find('sensor.other.radar')
    rad_bp.set_attribute('horizontal_fov', str(125))
    rad_bp.set_attribute('vertical_fov', str(115))
    rad_bp.set_attribute('range', str(285))
    rad_bp.set_attribute('points_per_second', str(2000))
    rad_location = carla.Location(x=2.25, z=0.9)
    rad_rotation = carla.Rotation()
    rad_transform = carla.Transform(rad_location,rad_rotation)
    return world.spawn_actor(rad_bp,rad_transform,attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)


In [8]:
def spawn_walker():
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

    start_location = carla.Location(x=70, y=25, z=1)

    trans = carla.Transform()
    trans.location = start_location

    return world.spawn_actor(walker_bp, trans)

# **FUNZIONI DI SUPPORTO GENERICHE**

In [9]:
# Convertitori
def check_lateral_collision(azimuth, depth):
    global vehicle_lateral_dimension
    return abs(depth * math.sin(azimuth)) < vehicle_lateral_dimension

def check_vertical_collision(altitude, depth):
    global vehicle_vertical_dimension
    return abs(depth * math.sin(altitude)) < vehicle_vertical_dimension 

def get_projected_velocity(azimuth, altitude, velocity):
    return abs(math.cos(azimuth) * math.cos(altitude) * velocity)

def get_projected_depth(azimuth, altitude, depth):
    return abs(math.cos(azimuth) * math.cos(altitude) * depth)

def get_breaking_distance(projected_velocity):
    global asphalt_friction_deceleration
    return (1/2) * projected_velocity**2 / asphalt_friction_deceleration

def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

In [10]:
# Comandi guida veicolo
def move_forward(vehicle):
    global throttle
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)
    while True:
        if get_vehicle_velocity(vehicle) < 1:
            break

In [11]:
# Simulatore sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation
    weather.precipitation_deposits
    weather.wetness
    weather.dust_storm
    return 0.8

# **LOGICA ADAS**

In [12]:
def forward_collision_callback(radar_data, vehicle):
  global min_fcw_state, counter, min_ttc, velocity_threshold, average_reaction_time
  for detection in radar_data:
    vehicle_velocity = get_vehicle_velocity(vehicle)
    if vehicle_velocity > velocity_threshold:
      if detection.velocity < 0 and check_lateral_collision(detection.azimuth, detection.depth) and check_vertical_collision(detection.altitude, detection.depth):
        projected_velocity = get_projected_velocity(detection.azimuth, detection.altitude, detection.velocity)   
        breaking_distance = (projected_velocity)
        projected_depth = get_projected_depth(detection.azimuth, detection.altitude, detection.depth)
        reacting_distance = max(0, projected_depth - breaking_distance)
        ttc = reacting_distance / projected_velocity
        fcw_state = Fcw_state.IDLE
        if ttc < min_ttc:
          if vehicle_velocity < projected_velocity - 2*velocity_threshold:
            fcw_state = Fcw_state.ESCAPE
          else:
            fcw_state = Fcw_state.ACTION
        elif ttc < min_ttc + average_reaction_time:
          fcw_state = Fcw_state.WARNING
        if fcw_state.value < min_fcw_state.value:
          min_fcw_state = fcw_state    
    else:
      fcw_state = Fcw_state.IDLE
        

# **SIMULAZIONE**

In [ ]:
try:
    vehicle = spawn_vehicle()
    radar = spawn_radar(vehicle)
    walker = spawn_walker()
    #vehicle.set_autopilot(True)
    radar.listen(lambda radar_data: forward_collision_callback(radar_data, vehicle))
    while True:
        if min_fcw_state == Fcw_state.ACTION:
            stop_vehicle(vehicle)
            print("Emergency braking activated!")  
            time.sleep(5)
            break
        elif min_fcw_state == Fcw_state.WARNING:    
            print("Emergency warning!")  
        elif min_fcw_state == Fcw_state.ESCAPE: 
            print("EscapeTime!") 
        else: 
            move_forward(vehicle)
        asphalt_friction_coefficient = get_asphalt_friction_coefficient()
        asphalt_friction_deceleration = 9.81 * asphalt_friction_coefficient
        time.sleep(0.05)

finally:
    vehicle.destroy()
    radar.destroy()
    walker.destroy()
    